# Project 3

In this project, we apply machine learning methods to predict Consumer Price Index. 

After obtaining the predicted CPI, we would then calculate monthly and yearly inflation.

After carefully considering the underlying structure of the data, we decided to build models using the period 2010-2020

- 2010 - 2017 as training data

- 2017 - 2019 as validation data

- 2019 - 2020 as test data


# I. Preprocessing 

## 1. Label Decomposition

Import necessary library

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, acf, pacf
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.arima.model import ARIMA
# from pandas_profiling import ProfileReport

In [ ]:
# Read in the data

df = pd.read_csv('cpi.csv', parse_dates= [['Year', 'Month']], index_col= 'Year_Month')

# get data from 2010 to 2020
df = df.loc['2010-01-01':'2019-12-31']

# Set the monthly frequency for the data

df.index.freq = 'MS'

# Change the index name to 'Date'
df.index.name = 'Date'

Visualize monthly and yearly inflation

In [ ]:
df['1-Month % Change'].plot()
plt.title('1-month inflation rate')

In [ ]:
df['12-Month % Change'].plot()
plt.title('12-month inflation rate')

Our current main focus is the CPI index, so let's decompose this feature first.
- First, decompose the CPI column into trend, seasonal, and residual components using additive method. 
- Second, apply multiplicative method
- Since we may apply detrending method as a way to make the data stationary, we will be using backward looking moving average in order to smooth out the noise (instead of the center moving average) and reduce the number of future observation lost. 


In [ ]:
df['CPI'].describe()

### 1.1 Additive decomposition

In [ ]:
additive_decomposed = seasonal_decompose(df['CPI'], model='additive',two_sided= False, period= 6)

# Plot the original data, trend, seasonal, and residual components
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(10, 8), sharex=True)

# Original data
ax1.plot(df['CPI'])
ax1.set_title('Original Data')
ax1.grid()

# Trend component
ax2.plot(additive_decomposed.trend)
ax2.set_title('Trend Component')
ax2.grid()

# Seasonal component
ax3.plot(additive_decomposed.seasonal)
ax3.set_title('Seasonal Component')
ax3.grid()

# Residual component
ax4.plot(additive_decomposed.resid)
ax4.set_title('Residual Component')
ax4.grid()

plt.tight_layout()
plt.show()


A statistical look into the seasonal component

In [ ]:
additive_decomposed.seasonal.describe()

### 1.2 Multiplicative Decomposition

In [ ]:
multiplicative_decomposed = seasonal_decompose(df['CPI'], model='multiplicative',two_sided= False, period= 6)

# Plot the original data, trend, seasonal, and residual components
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(10, 8), sharex=True)

# Original data
ax1.plot(df['CPI'])
ax1.set_title('Original Data')
ax1.grid()

# Trend component
ax2.plot(multiplicative_decomposed.trend)
ax2.set_title('Trend Component')
ax2.grid()

# Seasonal component
ax3.plot(multiplicative_decomposed.seasonal)
ax3.set_title('Seasonal Component')
ax3.grid()

# Residual component
ax4.plot(multiplicative_decomposed.resid)
ax4.set_title('Residual Component')
ax4.grid()

plt.tight_layout()
plt.show()


### 1.3 Decomposition Conclusion

- After trying multiple periods/frequencies, we decided to use a period of 6 to decompose the CPI index as it results the perfect seasonal component. 

Both multiplicative and additive decomposition show that the trend component is the most important component in the CPI index. 

However, residuals in the multiplicative decomposition is more stable than in additive approach, so we should move forward with mulitplicative approach. 

## 2. Trend Analysis

In [ ]:
# Obtain statistical attributes of the trend component
additive_decomposed.trend.describe()

Since the series has a linear trend, it is definitely not stationary. Thus, we should attempt to make it stationary.

In addition, we can address how statistical properties of a series change over time by visualizing. This would help us check the structural break and heteroscedasticity issue. 
- The rolling window size is 12 months

In [ ]:
# Create a fucntion to plot rolling variance and rolling mean
def rolling_statistics(timeseries, custom_name, window_size=12):
    # Determine rolling statistics
    rolling_mean = timeseries.rolling(window=window_size).mean()
    rolling_std = timeseries.rolling(window=window_size).std()

    # Plot rolling statistics
    plt.figure(figsize=(10, 6))
    plt.plot(rolling_mean, color='black', label='Rolling Mean')
    plt.plot(rolling_std, color='red', label='Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation of ' + custom_name)
    plt.grid()
    plt.show()

### 2.1 Label Diffencing

First, let's work on differncing the data to see if the process can make the data more stationay. 

#### 2.1.1 First Order Differencing

In the first order differencing, we would subtract the immediate previous value from the current value to obtain the difference between two consecutive periods. 

First-Order Differencing = Value at time t - Value at time t-1

In [ ]:
diff_data = df['CPI'].diff().dropna()


In [ ]:
diff_data.plot()
plt.title('First - Order Differenced Data')

In [ ]:
rolling_statistics(diff_data, 'First - Order Differenced Data')

#### 2.1.2 Second Order Differencing

In [ ]:
second_order_diff = diff_data.diff().dropna()

In [ ]:
second_order_diff.plot()
plt.title('Second - Order Differenced Data')


In [ ]:
rolling_statistics(second_order_diff, 'Second - Order Differenced Data')

### 2.2 Label Detrending

- The method for smoothing data used in this project is backward moving average.

- Detrended data is computed by subtracting the trend values from the actual values. 

- Since we use a period of 6 to smooth out the data, the function will use a centered moving average witha window size of 6 to smooth the trend component (6 periods prior to the current value).

- As a result, we would lose 6 observations in using label detrending, compared to only 1 in first-order differencing, and 2 in second-order differencing.

In [ ]:
# Here, I extract the trend component from the multiplicative decomposition. Trend values from either multiplicative or additive decompositions are identical.
trend = multiplicative_decomposed.trend

In [ ]:
detrend = df['CPI']- trend

detrend.dropna(inplace=True)

In [ ]:
detrend.plot()
plt.title('Detrended Data')

In [ ]:
rolling_statistics(detrend, 'Detrended Data')

### 2.3 Differencing and Detrending Conclusion

- Mean and variance of these transformed data are not constant over time. Between the 3 transformation method, the second order differencing appear to be the most stationary. Therefore, we would move forward with second order differencing.

## 3. Label Transformation (Make it stationary)

Create a function to calculate the ADF test and print out the result. 

In [ ]:
def stationary_test(input):

    result = adfuller(input)
    print('ADF Statistic:', result[0])
    print('p-value:', result[1])
    print('Critical Values:', result[4])

    # Reject the null hypothesis if the p-value is below the chosen significance level
    if result[1] < 0.05:
        print("The data is STATIONARY.")
    else:
        print("The data is NOT STATIONARY.")
        

In addition to the ADF test, let's use the non parametric KPSS test to confirm the stationarity of the data. If KPSS's result contradict conclusion from ADF, we need to investigate further. 

### 3.1 Augemnted Dickey-Fuller Test

To statistically verify if the data is stationary or not, we would deploy ADF test. 

- Null hypothesis: The time series contains a unit root and is non-stationary

- Alternative hypothesis is that the time series is stationary. 

To confirm that the data is stationary, we need a p-value that is lower than the significance level in order to reject the null hypothesis, and the critical values should be greater greater than the ADF statistics.

- The significance level chosen is 0.05. 

1. ADF on the orignal dataset

In [ ]:
stationary_test(df['CPI'])

2. ADF on the second order differenced dataset

In [ ]:
stationary_test(second_order_diff)

### 3.2 Non-parametric KPSS test

- Null hypothesis: The time series is stationary (no unit root)

- Alternative hypothesis: The time series is stastionary (it has a unit root)

KPSS' test statistic is compared to the relevant critical values. If the test statistic is greater than the cirtical value at a chosen level of significance, we reject the null hypothesis  and conclude that the series is non-stationary with a unit root. 


In [ ]:
# Create a function to perform the kpss test.
def kpss_test(input):
        result = kpss(input)
        print('KPSS Statistic:', result[0])
        print('p-value:', result[1])
        print('Critical Values:', result[3])
    
        # Reject the null hypothesis if the p-value is below the chosen significance level
        if result[1] < 0.05:
            print("The data is NOT STATIONARY.")
        else:
            print("The data is STATIONARY.")


1. KPSS test on the orignal data

In [ ]:
kpss_test(df['CPI'])

2. KPSS test on the second-order differenced data

In [ ]:
kpss_test(second_order_diff)

Most critical values across level of significance are well beyond the test statistic. This supports the Null hypothesis that the series is stationary

### 3.3 ADF and KPSS test conclusion 

The second-order differencing data is found to be stationary by using ADF and KPSS test, while the original data is not stationary (as expected). Results from both test are consistent.

In [ ]:
# Create a box plot to compare the distribution of the detrended data and the first-order differenced data
def cus_boxplot(data1, title1):
    fig, ax1 = plt.subplots(1, 1, figsize=(5, 5))
    sns.boxplot(data1, ax=ax1)
    ax1.set_title(title1)
    plt.show()

In [ ]:
cus_boxplot(second_order_diff, 'Second-Order Differenced Data')

In [ ]:
# Get the data statisitcal summary

print('Second order difference data statistical summary:')
second_order_diff.describe()

### 3.3 White Noise Check 

In this test, we would test the autocorrelation between the current value its 12 lags. If there exist a correlation between the current value and a number of its lags, then the series is not white noise

In [ ]:
# Create a function to check if a pandas time series is a white noise. Import package for acirr_ljungbox test
from statsmodels.stats.diagnostic import acorr_ljungbox
def white_noise_test(input):
    # Calculate the p-value of the autocorrelation
    lags = 12
    p_val_list = []
    for i in range(1, lags):
        result = acorr_ljungbox(input, lags= lags)
        p_value = result.iloc[i-1,1]
        p_val_list.append(p_value)
    # check if all p_values in the list are below 0.05, then the time series is not a white noise
    if all(i < 0.05 for i in p_val_list):
        print('The time series is NOT a white noise.')
    

In [ ]:
white_noise_test(second_order_diff)

Since the series illustrate a correlation between the current value and its lags, the data is thus not white noise. 

## 4. Lag Analysis

To identify the useful lag variables, we can use the autocorrelation function (ACF) and Partial Autocorrelation Function (PACF) plots.

The main difference between ACF and PACF is that ACF measures the total correlation between a time series and its lagged values, while PACF measures the direct correlation between a time series and its lagged values after removing the effect of the correlations with the intervening observations. 

ACF is primarily used to determine the MA component, while the PACF plot is used to determine the AR component.

The shaded area is the signifiance level in the ACF and PACF plots. If a lag is above the shaded area, it is significantly correlated with the label. 

### 4.1 Label's ACF and PACF

In [ ]:
# ACF plot
plot_acf(second_order_diff, lags= 24, zero=False)
plt.title('ACF Plot of Second-Order Differenced Data')
plt.show()

# PACF plot
plot_pacf(second_order_diff, lags = 24, zero=False)
plt.title('PACF Plot of Second-Order Differenced Data')
plt.show()

### 4.2 Lag Analysis Conclusion 

- The ACF plot shows that the label is correlated with its lagged values up to 3 periods.

- Meanwhile, the PACF shows that the label is directly correlated with the first 4 lag values and lags of 9 and 22. We can't really be sure that lag 22 are really substantially significnnt as it shows on the graph due to the small size of the data.

## 5. Splitting the data

In [ ]:
# Training, validation, and test sets

train = second_order_diff.loc['2010-01-01':'2016-12-31']

val = second_order_diff.loc['2017-01-01':'2018-12-31']

test = second_order_diff.loc['2019-01-01':'2019-12-31']


# II. Modeling 1 (Lag Predictors only)

## 1. Base model: ARIMA(1,2,1)

- The ARIMA(p,d,q) model contains 3 main components: AR, I (differencing), and MA.

- After carefully taking into consideration, second-order differencing seems to be the best way to make the data stationary so decided to use it as the base model for comparision purpose.

- The model takes into account 1 lagged values, 1 lagged errors, and 2 order differencing. 

### 1.1 Model Executing

In [ ]:
# Create and fit an ARIMA(1,2,1) model to the training set

#! Here we set I = 0 since we have manually differenced the data
base_model = ARIMA(train, order=(1,0,1)).fit()


### 1.2 Model Summary

In [ ]:
base_model.summary()

- The lag of 1 component is found statistically insignificant since it has a very high p-value. Meanwhile, the AR component, which is the error term of the 1st lag. 

- The negative figure for skew and kurtosis also tell us about the distribution of the model's residuals as they are found to be skewed to the left and contain a fat tail. 

### 1.3 Predicting the Validation set

- if possible, please repeat the mean, standard deviation of the label here (2nd-order differenced)

In [ ]:
# Forecast values for the validation set
validation_forecast = base_model.forecast(steps=len(val))

In [ ]:
# Plot the forecasted values and the actual values
plt.figure(figsize=(10, 6))
plt.plot(val, label='Actual')
plt.plot(validation_forecast, label='Forecast')
plt.legend(loc='upper left')
plt.title('ARIMA(1,0,1)')


### 1.4 Model Evaluation

In [ ]:
# Calculate evaluation metrics
mae = np.mean(np.abs(validation_forecast - val))
mse = np.mean((validation_forecast - val)**2)
rmse = np.sqrt(mse)

# Print evaluation metrics
print(f"MAE: {mae:.2f}, MSE: {mse:.2f}, RMSE: {rmse:.2f}")

## 2. ARIMA with more ARs and MAs

From ACF and PACF results above, we were able to identify lags that are significantly correlated with the label, 

- ACF's result is helpful in determining AR components, while PACF's helps determine MA components

From the graphs earlier, we would sequtially add MA and AR component to the model and observe how AIC and BIC change.

- A lower BIC and AIC are preferred. 

Let's write a for loop to loop through the potential models and view the results. 

In [ ]:
ar = [2,3]
ma = [1,2,3,4]

for i in ma:

    for j in ar:
        
        # train and fit the model
        
        model = ARIMA(train, order=(j,0,i)).fit(method_kwargs={'maxiter': 100})
        
        validation_forecast = model.forecast(steps=len(val))
        
        # Calculate evaluation metrics
        
        mae = np.mean(np.abs(validation_forecast - val))
        
        mse = np.mean((validation_forecast - val)**2)
        
        rmse = np.sqrt(mse)
        
        # Plot the forecasted values and the actual values
        
        plt.figure(figsize=(10, 6))
        
        plt.plot(val, label='Actual')
        
        plt.plot(validation_forecast, label='Forecast')
        
        plt.legend(loc='upper left')
        
        plt.title(f'ARIMA({j},0,{i})')
        
        plt.text(0.88, 0.98, f'MAE: {mae:.2f}\nMSE: {mse:.2f}\nRMSE: {rmse:.2f}', 
                 transform=plt.gca().transAxes, verticalalignment='top')
        
        plt.show()

        # Attacht the model's summary right below the graph

        print(model.summary())
      

## 2. ARIMA Model's Conclusion

- The best ARIMA model so far is ARIMA(3,0,3). For some other ARIMA model versions, the maximum likelihood optimization method fails to converge. Therefore, it leads to poor predictions, as we can see there is a horizontal line for some ARIMA model's predictions, which is completely different than the ARIMA(3,0,3)

# III. Engineearing Models With Ext Components 

## 1. Preprocessing Predictors 

### 1.1 Import and format data

First, we need to import data with external predictors 

In [ ]:
predictors = pd.read_csv('full_data.csv', index_col='Date', parse_dates=True)

In [ ]:
# Get some basic infor from the data 
predictors.describe().round(2)

In [ ]:
# Make the date consistent with the CPI data
predictors = predictors.loc[:'2019-12-31']

### 1.2 Apply first order differencing on predictors

- Since we have taken differencing on CPI, it makes sense to take transform predictors to at least a first order differencing as well. Also, we would like to see how the change in these variables affect movement in the label.
- Also, as I have attempted to use the original data, the multicollinarity issue was so serious that we can't move forward with it.



In [ ]:
# apply diff on all columns in predictors 
predictors = predictors.diff().dropna()

### 1.3 Normalize Predictors 

#### 1.3.1 Remove Outliers

All predictors are deemed to be equally important but they appear to be on different scale, thus 

In [ ]:
# Visualize the data by plotting their distributions and boxplots
# sns.pairplot(predictors)

In [ ]:
# Replace all outliers in the predictors file using IQR method
def replace_outliers(data):
    for col in data.columns:
        q1 = data[col].quantile(0.25)
        q3 = data[col].quantile(0.75)
        iqr = q3 - q1
        
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        
        data[col] = np.where(data[col] < lower_bound, lower_bound, data[col])
        data[col] = np.where(data[col] > upper_bound, upper_bound, data[col])
    return data


In [ ]:
clean_predictors = replace_outliers(predictors)

#### 1.3.2 Normalize predictors

In [ ]:
# Normalize clean predictors data using min-max scaler, and convert it to a dataframe
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
clean_predictors = scaler.fit_transform(clean_predictors)
clean_predictors = pd.DataFrame(clean_predictors, columns=predictors.columns, index=predictors.index)

### 1.4 Merge with the label

In [ ]:
# Merge the clean predictors data with the CPI data
full_data = pd.merge(second_order_diff, clean_predictors, left_index=True, right_index=True)


## 2. Correlation Analysis

In [ ]:
# Calculate the correlation matrix of full_data and visualize it using heatmap
corr_matrix = full_data.corr().round(2)
plt.figure(figsize=(15, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.show()

### 2.1 Correlation Analysis Conclusion

- Most features are moderately or weakly correlated with CPI. In economic sense, they should have a strong correlation with the label, however, since we have differenced both label and features, the strong correlation no longer holds. 

- Though some features like Money_Stock (M2 money supply) and FedSurDef are found to have a small correlation with the label, it might still be useful based on our domain knowledge. 
 
- In addition, since correlation measures only linear relationships, non-linear relationships between predictors and lable can still be significant and useful for prediction and they won't be captured by correlation coefficients. 


## 3. Feature Selection with Lasso Regression

- Though the current set of variables look good. Next, we apply Lasso Regression to filter the number of predictors even further in order to retain the most important variables only. 

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LassoCV
# split the data 
target = 'CPI'

train = full_data.loc['2010-01-01':'2016-12-31']

val = full_data.loc['2017-01-01':'2018-12-31']

test = full_data.loc['2019-01-01':'2019-12-31']

x_train = train.drop(columns = [target])

y_train = train[target]

x_val = val.drop(columns = [target])

y_val = val[target]

The best alpha as performed below is the one that provides the optimal balance between fitting the data and preventing overfitting. 

In [ ]:
# Create and fit a lasso regression with cross validation to find the best alpha
model = LassoCV(alphas = None, cv = 3, random_state=123).fit(x_train, y_train)

best_alpha = model.alpha_

print(f"Best alpha: {best_alpha:.4f}")

- Though we have found the best alpha, we are unable to apply it to the lasso regresion since it would only keep Crude oil as the sole predictor for the model. 

- Therfore, we reduce alpha to 0.01, while maintaining the same RMSE but it include more predictors for the model.

In [ ]:
# Now we can fit the model with the best alpha
final_lasso = Lasso(alpha=0.01, random_state=123).fit(x_train, y_train)

In [ ]:
# Evaluate the model performance on the validation set 
val_predictions = final_lasso.predict(x_val)
val_mse = mean_squared_error(y_val, val_predictions)
val_rmse = np.sqrt(val_mse)
print(f'Validation RMSE: {val_rmse:.2f}\n')

In [ ]:
# Insepct the coefficients to see which predictors were retained in the model 
coef_df = pd.DataFrame({'Feature': x_train.columns, 'Coefficient': final_lasso.coef_})
coef_df = coef_df.sort_values(by='Coefficient', ascending=False)
# print Feature from coef_df where Coefficient is different from 0

print('Here is the list of predictors that were retained in the lasso regression using alpha = 0.01')

coef_df[coef_df['Coefficient'] != 0]

In [ ]:
# extract a vector names for these retained variables. 
selected_predictors = coef_df[coef_df['Coefficient'] != 0]['Feature'].values

## 4. Random Forest

In [ ]:
# Import necessary libraries for random forest regression 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

Create a base random forest regression model

In [ ]:
rf_base = RandomForestRegressor(random_state=123)

# Train the base model 
rf_base.fit(x_train, y_train)

Define hyperparameter search space for grid search or random search:

In [ ]:
# Hyperparameter search space
param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [10, 20, 30, 50, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False],
}

Chooose a search method (GridSearchCV or RandomizedSearchCV) and fit the model:

In [ ]:
# Grid search

grid_search = GridSearchCV(estimator=rf_base, 
    param_grid=param_grid, 
    cv=3, 
    n_jobs=-1, 
    verbose=2)

# Random search
# n_iter: Number of random parameter combinations to try
random_search = RandomizedSearchCV(estimator=rf_base, 
    param_distributions=param_grid, 
    n_iter=100, 
    cv=5, 
    n_jobs=-1, 
    verbose=2, 
    random_state=123)

# Fit the search object, here we can use either random search or grid searchq

grid_search.fit(x_train, y_train)

random_search.fit(x_train, y_train)

Get the best hyperparameters from the search

In [ ]:
grid_search_params = grid_search.best_params_

random_search_params = random_search.best_params_

print(f'Grid Search Best Hyperparameters: {best_hyperparameters}')

print(f'Random Search Best Hyperparameters: {random_search_params}')

Both GridSearchCV and RandomizedSearchCV returned the same best hyperparameters. 

Train the random forest model with the best hyperparameters

In [ ]:
# Instantiate the model with the best hyperparameters
best_rf_regressor = RandomForestRegressor(**grid_search_params, random_state=123)

# Train the model 
best_rf_regressor.fit(x_train, y_train)

Make predicitons and evaluate the model performance using RMSE

In [ ]:
# Make predictions
y_pred = best_rf_regressor.predict(x_val)

# Evaluate the model 

mse = mean_squared_error(y_val, y_pred)

rmse = np.sqrt(mse)

mae = np.mean(np.abs(y_pred - y_val))

In [ ]:
# Plot y_val and y_pred on the same graph, but first, we need to add a time index to y_pred
y_pred = pd.Series(y_pred, index=y_val.index)
plt.figure(figsize=(10, 6))
plt.plot(y_val, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend(loc='upper left')
plt.title('Random Forest Regression')
plt.text(0.88, 0.98, f'MAE: {mae:.2f}\nMSE: {mse:.2f}\nRMSE: {rmse:.2f}', 
                 transform=plt.gca().transAxes, verticalalignment='top')
plt.show()

# ideas for tomorrow. 

- add lag as a variable to random forest, probably from ARIMA(3,0,3)